# MNIST 手写数字分类完整实战

## 概述

本 Notebook 系统性地介绍分类任务的核心概念与实践方法，涵盖：

- **二分类器**：使用 SGD 训练单数字识别模型
- **性能评估**：交叉验证、混淆矩阵、精确率、召回率、F1 分数
- **阈值调整**：Precision-Recall 曲线、ROC 曲线与 AUC
- **多分类策略**：OvR (One-vs-Rest)、OvO (One-vs-One)
- **多标签分类**：同时预测多个属性

## 数据集

MNIST (Modified National Institute of Standards and Technology) 是机器学习领域最经典的基准数据集之一：
- 70,000 张 28×28 灰度手写数字图像
- 训练集 60,000 张，测试集 10,000 张
- 10 个类别 (0-9)

## 环境要求

```
scikit-learn >= 1.0
numpy >= 1.20
matplotlib >= 3.5
```

---

## 1. 环境配置与数据加载

In [ ]:
# =============================================================================
# 标准库与第三方库导入
# =============================================================================
import warnings
warnings.filterwarnings('ignore')

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl

# scikit-learn 模块
from sklearn.datasets import fetch_openml
from sklearn.model_selection import (
    StratifiedKFold, 
    cross_val_score,
    cross_val_predict
)
from sklearn.base import BaseEstimator, clone
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.multiclass import OneVsOneClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import (
    confusion_matrix,
    precision_score,
    recall_score,
    f1_score,
    precision_recall_curve,
    roc_curve,
    roc_auc_score
)

# 设置随机种子确保可复现性
RANDOM_STATE = 42
np.random.seed(RANDOM_STATE)

# Matplotlib 中文显示配置
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei', 'DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("环境配置完成")

In [ ]:
# =============================================================================
# 数据加载
# =============================================================================
# fetch_openml 从 OpenML 仓库获取标准数据集
# MNIST 784 表示每张图片被展平为 784 维向量 (28 × 28 = 784)
print("正在加载 MNIST 数据集...")
mnist = fetch_openml('mnist_784', version=1, as_frame=True, parser='auto')

X, y = mnist['data'], mnist['target']

print(f"数据集键: {mnist.keys()}")
print(f"特征矩阵形状: {X.shape}")
print(f"标签向量形状: {y.shape}")
print(f"标签类型: {y.dtype}")

In [ ]:
# =============================================================================
# 数据可视化：查看单个样本
# =============================================================================
def plot_digit(data, label=None, ax=None):
    """
    可视化单个手写数字图像
    
    Parameters
    ----------
    data : array-like, shape (784,)
        展平的图像数据
    label : str, optional
        图像标签
    ax : matplotlib.axes.Axes, optional
        绑定的坐标轴对象
    """
    if ax is None:
        fig, ax = plt.subplots(figsize=(3, 3))
    
    image = np.array(data).reshape(28, 28)
    ax.imshow(image, cmap='binary', interpolation='nearest')
    ax.axis('off')
    if label is not None:
        ax.set_title(f'Label: {label}', fontsize=12)

# 展示第一个样本
some_digit = X.iloc[0].values
plot_digit(some_digit, label=y.iloc[0])
plt.show()

In [ ]:
# =============================================================================
# 数据可视化：查看多个样本
# =============================================================================
def plot_digits(instances, labels, images_per_row=10):
    """
    批量可视化手写数字图像
    
    Parameters
    ----------
    instances : array-like
        图像数据集
    labels : array-like
        对应标签
    images_per_row : int
        每行显示的图像数量
    """
    n_images = len(instances)
    n_rows = (n_images + images_per_row - 1) // images_per_row
    
    fig, axes = plt.subplots(n_rows, images_per_row, 
                             figsize=(images_per_row * 1.2, n_rows * 1.4))
    axes = axes.flatten() if n_rows > 1 else [axes] if n_rows == 1 and images_per_row == 1 else axes
    
    for idx, ax in enumerate(axes):
        if idx < n_images:
            image = np.array(instances[idx]).reshape(28, 28)
            ax.imshow(image, cmap='binary', interpolation='nearest')
            ax.set_title(f'{labels[idx]}', fontsize=9)
        ax.axis('off')
    
    plt.tight_layout()
    plt.show()

# 展示前 30 个样本
sample_indices = range(30)
plot_digits(
    X.iloc[sample_indices].values, 
    y.iloc[sample_indices].values,
    images_per_row=10
)

In [ ]:
# =============================================================================
# 数据预处理与划分
# =============================================================================
# 将标签转换为整数类型
y = y.astype(np.uint8)

# MNIST 数据集已预先排序：前 60,000 为训练集，后 10,000 为测试集
# 注意：在实际项目中应使用 train_test_split 进行随机划分
X_train, X_test = X[:60000], X[60000:]
y_train, y_test = y[:60000], y[60000:]

print(f"训练集: {X_train.shape[0]} 样本")
print(f"测试集: {X_test.shape[0]} 样本")
print(f"类别分布 (训练集): \n{y_train.value_counts().sort_index()}")

---

## 2. 二分类器训练

### 理论背景

**二分类 (Binary Classification)** 是最基本的分类任务，将样本划分为两个互斥类别。

我们先构建一个简单任务：**识别数字 5**
- 正类 (Positive): 数字 5
- 负类 (Negative): 非数字 5

In [ ]:
# =============================================================================
# 构建二分类标签
# =============================================================================
# 布尔索引：True 表示正类 (数字5)，False 表示负类 (非数字5)
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

print(f"训练集正类比例: {y_train_5.sum() / len(y_train_5):.2%}")
print(f"测试集正类比例: {y_test_5.sum() / len(y_test_5):.2%}")
print("\n注意：这是一个典型的类别不平衡问题，正类仅占约 10%")

### 2.1 随机梯度下降分类器 (SGDClassifier)

**SGD (Stochastic Gradient Descent)** 是一种高效的优化算法：

- 每次只使用一个样本（或小批量）更新模型参数
- 适合处理大规模数据集
- 默认使用 Hinge Loss（等价于线性 SVM）

**关键超参数**：
- `max_iter`: 最大迭代次数
- `tol`: 收敛容忍度
- `random_state`: 随机种子

In [ ]:
# =============================================================================
# 训练 SGD 分类器
# =============================================================================
sgd_clf = SGDClassifier(
    max_iter=1000,      # 最大迭代次数
    tol=1e-3,           # 收敛容忍度
    random_state=RANDOM_STATE
)

# 训练模型
sgd_clf.fit(X_train, y_train_5)

# 对单个样本进行预测
prediction = sgd_clf.predict([some_digit])
print(f"样本真实标签: {y.iloc[0]}")
print(f"模型预测 (是否为5): {prediction[0]}")

---

## 3. 模型性能评估

### 3.1 交叉验证

**K 折交叉验证** 是评估模型泛化能力的标准方法：

1. 将数据划分为 K 个子集
2. 每次用 K-1 个子集训练，1 个子集验证
3. 重复 K 次，取平均性能

**分层抽样 (Stratified)**: 保持每折中类别比例与整体一致，对不平衡数据尤为重要。

In [ ]:
# =============================================================================
# 手动实现 K 折交叉验证（用于理解原理）
# =============================================================================
def manual_cross_validation(clf, X, y, n_splits=3):
    """
    手动实现分层 K 折交叉验证
    
    Parameters
    ----------
    clf : estimator
        scikit-learn 分类器
    X : array-like
        特征矩阵
    y : array-like
        标签向量
    n_splits : int
        折数
    
    Returns
    -------
    list : 每折的准确率
    """
    skfolds = StratifiedKFold(n_splits=n_splits, shuffle=True, random_state=RANDOM_STATE)
    scores = []
    
    for fold_idx, (train_idx, val_idx) in enumerate(skfolds.split(X, y), 1):
        # 克隆分类器，避免使用已训练的模型
        clf_clone = clone(clf)
        
        # 使用 iloc 进行索引（适配 DataFrame）
        X_train_fold = X.iloc[train_idx] if hasattr(X, 'iloc') else X[train_idx]
        y_train_fold = y.iloc[train_idx] if hasattr(y, 'iloc') else y[train_idx]
        X_val_fold = X.iloc[val_idx] if hasattr(X, 'iloc') else X[val_idx]
        y_val_fold = y.iloc[val_idx] if hasattr(y, 'iloc') else y[val_idx]
        
        # 训练与预测
        clf_clone.fit(X_train_fold, y_train_fold)
        y_pred = clf_clone.predict(X_val_fold)
        
        # 计算准确率
        accuracy = (y_pred == y_val_fold).sum() / len(y_val_fold)
        scores.append(accuracy)
        print(f"Fold {fold_idx}: Accuracy = {accuracy:.4f}")
    
    return scores

print("手动交叉验证结果:")
manual_scores = manual_cross_validation(sgd_clf, X_train, y_train_5, n_splits=3)
print(f"\n平均准确率: {np.mean(manual_scores):.4f} (+/- {np.std(manual_scores):.4f})")

In [ ]:
# =============================================================================
# 使用 scikit-learn 的 cross_val_score（推荐方式）
# =============================================================================
cv_scores = cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring='accuracy')
print(f"cross_val_score 结果: {cv_scores}")
print(f"平均准确率: {cv_scores.mean():.4f} (+/- {cv_scores.std():.4f})")

### 3.2 基准分类器：为什么准确率不够？

一个"永远预测为非5"的分类器也能达到 ~90% 准确率，因为只有约 10% 的样本是数字 5。

**结论**: 对于不平衡数据集，**准确率 (Accuracy)** 不是可靠的评估指标。

In [ ]:
# =============================================================================
# 构建基准分类器（永远预测为 False）
# =============================================================================
class Never5Classifier(BaseEstimator):
    """
    一个永远预测为负类的基准分类器
    
    用于演示：在类别不平衡时，准确率是一个误导性指标
    """
    def fit(self, X, y=None):
        return self
    
    def predict(self, X):
        return np.zeros((len(X),), dtype=bool)

# 评估基准分类器
never_5_clf = Never5Classifier()
dummy_scores = cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')

print(f"基准分类器准确率: {dummy_scores}")
print(f"平均: {dummy_scores.mean():.4f}")
print("\n结论: 一个什么都不做的分类器也能达到 ~90% 准确率！")

### 3.3 混淆矩阵 (Confusion Matrix)

混淆矩阵是评估分类器性能的基础工具：

```
                    预测
                Positive  Negative
实际  Positive    TP        FN
      Negative    FP        TN
```

- **TP (True Positive)**: 正确识别的正类
- **TN (True Negative)**: 正确识别的负类
- **FP (False Positive)**: 误报（Type I Error）
- **FN (False Negative)**: 漏报（Type II Error）

In [ ]:
# =============================================================================
# 计算混淆矩阵
# =============================================================================
# cross_val_predict 返回每个样本在作为验证集时的预测结果
y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)

# 生成混淆矩阵
conf_mat = confusion_matrix(y_train_5, y_train_pred)

print("混淆矩阵:")
print(conf_mat)
print(f"\n解读:")
print(f"  TN (真负): {conf_mat[0, 0]:,} - 正确识别的非5")
print(f"  FP (假正): {conf_mat[0, 1]:,} - 误判为5的非5")
print(f"  FN (假负): {conf_mat[1, 0]:,} - 误判为非5的5")
print(f"  TP (真正): {conf_mat[1, 1]:,} - 正确识别的5")

In [ ]:
# =============================================================================
# 完美分类器的混淆矩阵（用于对比）
# =============================================================================
perfect_conf_mat = confusion_matrix(y_train_5, y_train_5)  # 预测 = 真实
print("完美分类器的混淆矩阵:")
print(perfect_conf_mat)
print("\n特点: FP = 0, FN = 0")

### 3.4 精确率与召回率

**精确率 (Precision)**: 预测为正类的样本中，真正为正类的比例
$$\text{Precision} = \frac{TP}{TP + FP}$$

**召回率 (Recall / Sensitivity / TPR)**: 实际为正类的样本中，被正确识别的比例
$$\text{Recall} = \frac{TP}{TP + FN}$$

**权衡**:
- 高精确率 → 减少误报（宁可漏掉，不可误判）
- 高召回率 → 减少漏报（宁可误判，不可漏掉）

In [ ]:
# =============================================================================
# 计算精确率与召回率
# =============================================================================
precision = precision_score(y_train_5, y_train_pred)
recall = recall_score(y_train_5, y_train_pred)

print(f"精确率 (Precision): {precision:.4f}")
print(f"召回率 (Recall): {recall:.4f}")

# 手动验证计算
tp = conf_mat[1, 1]
fp = conf_mat[0, 1]
fn = conf_mat[1, 0]
print(f"\n手动计算:")
print(f"  Precision = TP/(TP+FP) = {tp}/({tp}+{fp}) = {tp/(tp+fp):.4f}")
print(f"  Recall = TP/(TP+FN) = {tp}/({tp}+{fn}) = {tp/(tp+fn):.4f}")

### 3.5 F1 分数

**F1 分数**是精确率与召回率的调和平均：
$$F_1 = 2 \cdot \frac{\text{Precision} \cdot \text{Recall}}{\text{Precision} + \text{Recall}}$$

**特点**:
- 只有当精确率和召回率都较高时，F1 才会高
- 适用于需要同时关注精确率和召回率的场景

In [ ]:
# =============================================================================
# 计算 F1 分数
# =============================================================================
f1 = f1_score(y_train_5, y_train_pred)
print(f"F1 分数: {f1:.4f}")

# 手动计算验证
f1_manual = 2 * (precision * recall) / (precision + recall)
print(f"手动计算: 2 × ({precision:.4f} × {recall:.4f}) / ({precision:.4f} + {recall:.4f}) = {f1_manual:.4f}")

---

## 4. 精确率/召回率权衡与阈值调整

SGDClassifier 使用决策函数计算分数：
- 分数 > 阈值 → 预测为正类
- 分数 ≤ 阈值 → 预测为负类

调整阈值可以控制精确率与召回率的权衡。

In [ ]:
# =============================================================================
# 决策函数与阈值
# =============================================================================
# 获取单个样本的决策分数
score = sgd_clf.decision_function([some_digit])
print(f"样本决策分数: {score[0]:.4f}")
print(f"默认阈值 (0) 下的预测: {score[0] > 0}")

# 使用不同阈值
thresholds_demo = [-10000, 0, 10000, 50000]
print("\n不同阈值下的预测:")
for t in thresholds_demo:
    print(f"  阈值 {t:>6}: {score[0] > t}")

In [ ]:
# =============================================================================
# 精确率-召回率曲线 (Precision-Recall Curve)
# =============================================================================
# 获取所有训练样本的决策分数
y_scores = cross_val_predict(
    sgd_clf, X_train, y_train_5, cv=3, method='decision_function'
)

# 计算不同阈值下的精确率和召回率
precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

# 绘制精确率/召回率 vs 阈值
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：指标随阈值变化
axes[0].plot(thresholds, precisions[:-1], 'b-', label='Precision', linewidth=2)
axes[0].plot(thresholds, recalls[:-1], 'g-', label='Recall', linewidth=2)
axes[0].set_xlabel('Threshold', fontsize=12)
axes[0].set_ylabel('Score', fontsize=12)
axes[0].set_title('Precision & Recall vs Threshold', fontsize=14)
axes[0].legend(loc='center right', fontsize=11)
axes[0].set_ylim([0, 1])
axes[0].grid(True, alpha=0.3)

# 右图：Precision-Recall 曲线
axes[1].plot(recalls, precisions, 'b-', linewidth=2)
axes[1].set_xlabel('Recall', fontsize=12)
axes[1].set_ylabel('Precision', fontsize=12)
axes[1].set_title('Precision-Recall Curve', fontsize=14)
axes[1].set_xlim([0, 1])
axes[1].set_ylim([0, 1])
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# =============================================================================
# 设定目标精确率，计算对应阈值
# =============================================================================
target_precision = 0.90

# 找到第一个达到目标精确率的阈值
threshold_90_precision = thresholds[np.argmax(precisions >= target_precision)]
print(f"达到 {target_precision:.0%} 精确率所需的阈值: {threshold_90_precision:.2f}")

# 使用新阈值进行预测
y_train_pred_90 = (y_scores >= threshold_90_precision)

# 评估新预测的指标
precision_90 = precision_score(y_train_5, y_train_pred_90)
recall_90 = recall_score(y_train_5, y_train_pred_90)

print(f"\n调整阈值后的性能:")
print(f"  精确率: {precision_90:.4f}")
print(f"  召回率: {recall_90:.4f}")
print(f"\n结论: 提高精确率会降低召回率，这是精确率-召回率权衡的体现")

### 4.1 ROC 曲线与 AUC

**ROC (Receiver Operating Characteristic) 曲线**:
- 横轴：假正例率 (FPR) = FP / (FP + TN)
- 纵轴：真正例率 (TPR) = TP / (TP + FN) = Recall

**AUC (Area Under Curve)**:
- AUC = 1.0: 完美分类器
- AUC = 0.5: 随机猜测
- AUC < 0.5: 比随机还差（通常意味着标签反了）

**PR 曲线 vs ROC 曲线**:
- 正类稀少 → 使用 PR 曲线
- 类别平衡 → 使用 ROC 曲线

In [ ]:
# =============================================================================
# 绘制 ROC 曲线
# =============================================================================
fpr, tpr, roc_thresholds = roc_curve(y_train_5, y_scores)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, 'b-', linewidth=2, label='SGD Classifier')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1, label='Random Classifier')
plt.xlabel('False Positive Rate (FPR)', fontsize=12)
plt.ylabel('True Positive Rate (TPR)', fontsize=12)
plt.title('ROC Curve', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.axis([0, 1, 0, 1])
plt.show()

# 计算 AUC
auc_score = roc_auc_score(y_train_5, y_scores)
print(f"ROC-AUC 分数: {auc_score:.4f}")

---

## 5. 模型对比：随机森林 vs SGD

In [ ]:
# =============================================================================
# 随机森林分类器
# =============================================================================
# 注意：随机森林使用 predict_proba 输出概率，而非 decision_function
forest_clf = RandomForestClassifier(
    n_estimators=100,      # 树的数量
    n_jobs=-1,             # 使用所有 CPU 核心
    random_state=RANDOM_STATE
)

# 获取预测概率
y_probas_forest = cross_val_predict(
    forest_clf, X_train, y_train_5, cv=3, method='predict_proba'
)

# 使用正类概率作为分数
y_scores_forest = y_probas_forest[:, 1]

# 计算 ROC 曲线
fpr_forest, tpr_forest, _ = roc_curve(y_train_5, y_scores_forest)

# 绘制对比图
plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, 'b:', linewidth=2, label=f'SGD (AUC={auc_score:.3f})')
plt.plot(fpr_forest, tpr_forest, 'g-', linewidth=2, 
         label=f'Random Forest (AUC={roc_auc_score(y_train_5, y_scores_forest):.3f})')
plt.plot([0, 1], [0, 1], 'k--', linewidth=1)
plt.xlabel('False Positive Rate', fontsize=12)
plt.ylabel('True Positive Rate', fontsize=12)
plt.title('ROC Curve Comparison', fontsize=14)
plt.legend(loc='lower right', fontsize=11)
plt.grid(True, alpha=0.3)
plt.show()

print(f"SGD AUC: {auc_score:.4f}")
print(f"Random Forest AUC: {roc_auc_score(y_train_5, y_scores_forest):.4f}")

---

## 6. 多分类器

将二分类扩展到多分类的两种主要策略：

1. **OvR (One-vs-Rest)**: 训练 K 个二分类器，每个区分"类别 k vs 其他"
2. **OvO (One-vs-One)**: 训练 K(K-1)/2 个二分类器，每个区分两个类别

In [ ]:
# =============================================================================
# SVM 多分类器
# =============================================================================
# 使用小批量数据进行快速演示
sample_size = 5000
X_train_small = X_train[:sample_size]
y_train_small = y_train[:sample_size]

print(f"使用 {sample_size} 个样本进行 SVM 训练演示")

# SVC 默认使用 OvO 策略
svm_clf = SVC(kernel='rbf', random_state=RANDOM_STATE)
svm_clf.fit(X_train_small, y_train_small)

# 预测
prediction = svm_clf.predict([some_digit])
print(f"\n样本真实标签: {y.iloc[0]}")
print(f"SVM 预测结果: {prediction[0]}")

# 查看决策分数
scores = svm_clf.decision_function([some_digit])
print(f"\n各类别决策分数:")
for i, score in enumerate(scores[0]):
    print(f"  类别 {i}: {score:.4f}")
print(f"\n最高分类别: {np.argmax(scores)} (与预测一致)")

In [ ]:
# =============================================================================
# 显式使用 OvO 策略
# =============================================================================
ovo_clf = OneVsOneClassifier(SVC(kernel='rbf', random_state=RANDOM_STATE))
ovo_clf.fit(X_train_small, y_train_small)

prediction = ovo_clf.predict([some_digit])
print(f"OvO 分类器预测: {prediction[0]}")
print(f"分类器数量: {len(ovo_clf.estimators_)} (10 个类别需要 45 个分类器)")

In [ ]:
# =============================================================================
# SGD 多分类（使用 OvR）
# =============================================================================
# 重新训练 SGD 分类器用于多分类
sgd_clf_multi = SGDClassifier(max_iter=1000, tol=1e-3, random_state=RANDOM_STATE)
sgd_clf_multi.fit(X_train, y_train)

# 查看决策分数
scores = sgd_clf_multi.decision_function([some_digit])
print("SGD 多分类决策分数:")
for i, score in enumerate(scores[0]):
    print(f"  类别 {i}: {score:>10.4f}")
print(f"\n预测类别: {np.argmax(scores[0])}")

In [ ]:
# =============================================================================
# 特征缩放对性能的影响
# =============================================================================
# 标准化：将特征缩放到均值为 0，标准差为 1
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train.astype(np.float64))

# 比较缩放前后的性能
print("特征缩放对 SGD 分类器的影响:")
scores_unscaled = cross_val_score(sgd_clf_multi, X_train, y_train, cv=3)
print(f"  未缩放: {scores_unscaled.mean():.4f} (+/- {scores_unscaled.std():.4f})")

scores_scaled = cross_val_score(sgd_clf_multi, X_train_scaled, y_train, cv=3)
print(f"  已缩放: {scores_scaled.mean():.4f} (+/- {scores_scaled.std():.4f})")
print("\n结论: 特征缩放显著提升了 SGD 分类器的性能")

---

## 7. 误差分析

In [ ]:
# =============================================================================
# 多分类混淆矩阵可视化
# =============================================================================
# 使用缩放后的数据进行预测
y_train_pred_multi = cross_val_predict(sgd_clf_multi, X_train_scaled, y_train, cv=3)
conf_mx_multi = confusion_matrix(y_train, y_train_pred_multi)

fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 左图：原始混淆矩阵
im1 = axes[0].matshow(conf_mx_multi, cmap=plt.cm.Blues)
axes[0].set_title('Confusion Matrix (Raw Counts)', fontsize=14, pad=20)
axes[0].set_xlabel('Predicted Label', fontsize=12)
axes[0].set_ylabel('True Label', fontsize=12)
plt.colorbar(im1, ax=axes[0])

# 右图：归一化后的错误率矩阵
# 按行归一化（每行代表一个真实类别）
row_sums = conf_mx_multi.sum(axis=1, keepdims=True)
norm_conf_mx = conf_mx_multi / row_sums

# 将对角线置零，只显示错误
np.fill_diagonal(norm_conf_mx, 0)

im2 = axes[1].matshow(norm_conf_mx, cmap=plt.cm.Reds)
axes[1].set_title('Error Rate Matrix (Diagonal Zeroed)', fontsize=14, pad=20)
axes[1].set_xlabel('Predicted Label', fontsize=12)
axes[1].set_ylabel('True Label', fontsize=12)
plt.colorbar(im2, ax=axes[1])

plt.tight_layout()
plt.show()

# 分析最常见的错误
print("最常见的分类错误 (错误率 > 5%):")
for i in range(10):
    for j in range(10):
        if i != j and norm_conf_mx[i, j] > 0.05:
            print(f"  {i} 被误判为 {j}: {norm_conf_mx[i, j]:.1%}")

---

## 8. 多标签分类

**多标签分类**: 每个样本可以同时属于多个类别

示例任务：
1. 该数字是否 ≥ 7？（大数字）
2. 该数字是否为奇数？

In [ ]:
# =============================================================================
# 多标签分类
# =============================================================================
# 构建多标签目标
y_train_large = (y_train >= 7)  # 大数字: 7, 8, 9
y_train_odd = (y_train % 2 == 1)  # 奇数: 1, 3, 5, 7, 9

# 合并为多标签矩阵
y_multilabel = np.c_[y_train_large, y_train_odd]

print("多标签数据形状:", y_multilabel.shape)
print(f"\n样本 0 (数字 {y_train.iloc[0]}):")
print(f"  是否 >= 7: {y_train_large.iloc[0]}")
print(f"  是否为奇数: {y_train_odd.iloc[0]}")

# 使用 KNN 进行多标签分类
# KNN 天然支持多标签输出
knn_clf = KNeighborsClassifier(n_neighbors=5, n_jobs=-1)
knn_clf.fit(X_train, y_multilabel)

# 预测
prediction = knn_clf.predict([some_digit])
print(f"\n对样本 0 的多标签预测: {prediction[0]}")
print(f"  [是否>=7, 是否为奇数]")

In [ ]:
# =============================================================================
# 多标签分类评估
# =============================================================================
# 使用小样本进行演示（KNN 预测较慢）
sample_size = 10000

y_train_knn_pred = cross_val_predict(
    knn_clf, 
    X_train[:sample_size], 
    y_multilabel[:sample_size], 
    cv=3
)

# 计算加权 F1 分数
f1_weighted = f1_score(
    y_multilabel[:sample_size], 
    y_train_knn_pred, 
    average='weighted'
)

print(f"多标签分类 F1 分数 (weighted): {f1_weighted:.4f}")

# 分别评估每个标签
for i, label_name in enumerate(['large (>=7)', 'odd']):
    f1_label = f1_score(
        y_multilabel[:sample_size, i], 
        y_train_knn_pred[:, i]
    )
    print(f"  {label_name}: F1 = {f1_label:.4f}")

---

## 总结

### 核心知识点

1. **二分类器**
   - SGDClassifier 适合大规模数据
   - 决策边界由阈值控制

2. **性能评估**
   - 准确率在不平衡数据上具有误导性
   - 混淆矩阵揭示具体错误类型
   - 精确率/召回率适合不同业务场景
   - F1 分数是精确率与召回率的平衡

3. **阈值调整**
   - PR 曲线适合正类稀少的场景
   - ROC-AUC 适合类别平衡的场景

4. **多分类策略**
   - OvR: 训练 K 个分类器
   - OvO: 训练 K(K-1)/2 个分类器

5. **工程实践**
   - 特征缩放对某些算法至关重要
   - 误差分析帮助定向改进模型

### 下一步

- 尝试其他算法（LogisticRegression, GradientBoosting）
- 超参数调优（GridSearchCV, RandomizedSearchCV）
- 特征工程（PCA, 数据增强）
- 深度学习方法（CNN）